In [1]:
import pandas as pd
import os
from tqdm import tqdm

# 원본 텍스트가 저장된 디렉토리
original_text_dir = '법령_통합텍스트(현역)'
all_laws_data = []

print(f"'{original_text_dir}' 폴더에서 원본 법령 텍스트를 로드합니다...")
for filename in tqdm(os.listdir(original_text_dir)):
    if filename.endswith('.txt'):
        law_name = filename[:-4]
        file_path = os.path.join(original_text_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        all_laws_data.append({
            '법령명': law_name,
            '내용': content
        })

df_laws = pd.DataFrame(all_laws_data)
# 전체 법령 이름 리스트를 미리 생성 (검색에 사용)
law_names_list = df_laws['법령명'].tolist()

print(f"\n데이터 로딩 완료: 총 {len(df_laws)}개 법령")

'법령_통합텍스트(현역)' 폴더에서 원본 법령 텍스트를 로드합니다...


100%|██████████| 5502/5502 [00:00<00:00, 6884.46it/s]


데이터 로딩 완료: 총 5502개 법령


In [2]:
edges = []
print("\n법령 간 인용 관계를 추출합니다... (시간이 소요될 수 있습니다)")

# 각 법령을 순회 (source)
for index, row in tqdm(df_laws.iterrows(), total=df_laws.shape[0]):
    source_name = row['법령명']
    source_text = row['내용']

    # 다른 모든 법령을 검색 (target)
    for target_name in law_names_list:
        # 자기 자신을 인용하는 경우는 제외
        if source_name == target_name:
            continue
        
        # 텍스트에서 다른 법령 이름이 언급된 횟수를 카운트
        count = source_text.count(target_name)
        
        # 한 번이라도 언급되었다면 엣지 리스트에 추가
        if count > 0:
            edges.append({
                'source': source_name,
                'target': target_name,
                'weight': count
            })

# 엣지 리스트를 데이터프레임으로 변환
df_edges = pd.DataFrame(edges)

print(f"\n총 {len(df_edges)}개의 인용 관계(엣지)를 발견했습니다.")
print(df_edges.sort_values(by='weight', ascending=False).head())


법령 간 인용 관계를 추출합니다... (시간이 소요될 수 있습니다)


100%|██████████| 5502/5502 [02:44<00:00, 33.42it/s]


총 74158개의 인용 관계(엣지)를 발견했습니다.
             source       target  weight
61524  지방세특례제한법 시행령      조세특례제한법     318
57196   조세특례제한법 시행령         소득세법     302
61525  지방세특례제한법 시행령  조세특례제한법 시행령     300
56895       조세특례제한법         소득세법     290
57148   조세특례제한법 시행령         법인세법     286


In [5]:
import networkx as nx
from pyvis.network import Network

# 방향성이 있는 그래프(DiGraph) 생성
G = nx.DiGraph()

# 모든 법령을 노드로 추가
G.add_nodes_from(law_names_list)

# 데이터프레임에서 엣지 정보를 읽어와 가중치가 있는 엣지로 추가
# (source, target, weight) 형태의 튜플 리스트로 변환하여 전달
edge_tuples = [tuple(x) for x in df_edges.to_numpy()]
G.add_weighted_edges_from(edge_tuples)

print(f"\n네트워크 생성 완료: 노드 {G.number_of_nodes()}개, 엣지 {G.number_of_edges()}개")

# Pyvis 네트워크 객체 생성
net = Network(height='800px', width='100%', notebook=True, directed=True, cdn_resources='in_line')

# NetworkX 그래프를 Pyvis 그래프로 변환
net.from_nx(G)

# 네트워크 시각화 옵션 설정
net.show_buttons(filter_=['physics'])
# net.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100) # 레이아웃 옵션

# HTML 파일로 저장 및 출력
output_filename = "law_citation_network.html"
net.show(output_filename)

print(f"\n'{output_filename}'으로 인터랙티브 네트워크 시각화가 저장되었습니다.")
print("생성된 html 파일을 웹 브라우저로 열어 확인해보세요!")


네트워크 생성 완료: 노드 5502개, 엣지 74158개
law_citation_network.html

'law_citation_network.html'으로 인터랙티브 네트워크 시각화가 저장되었습니다.
생성된 html 파일을 웹 브라우저로 열어 확인해보세요!


In [6]:
# 1. 연결 중심성 (Degree Centrality): 가장 많이 연결된 법령
# In-degree: 다른 법령으로부터 가장 많이 인용'되는' 법령
in_degree = G.in_degree(weight='weight')
df_in_degree = pd.DataFrame(in_degree, columns=['법령명', 'In-Degree']).sort_values(by='In-Degree', ascending=False)

print("\n\n--- 가장 많이 인용되는 법령 TOP 10 (In-Degree) ---")
print(df_in_degree.head(10).to_string(index=False))

# 2. 페이지랭크 (PageRank): 구글 검색처럼 중요도/영향력이 높은 법령
pagerank = nx.pagerank(G, weight='weight')
df_pagerank = pd.DataFrame(pagerank.items(), columns=['법령명', 'PageRank']).sort_values(by='PageRank', ascending=False)

print("\n\n--- 영향력이 가장 높은 법령 TOP 10 (PageRank) ---")
print(df_pagerank.head(10).to_string(index=False))

# 3. 매개 중심성 (Betweenness Centrality): 법령들을 연결하는 '허브/가교' 역할을 하는 법령
betweenness = nx.betweenness_centrality(G, weight='weight')
df_betweenness = pd.DataFrame(betweenness.items(), columns=['법령명', 'Betweenness']).sort_values(by='Betweenness', ascending=False)

print("\n\n--- '허브' 역할을 하는 법령 TOP 10 (Betweenness) ---")
print(df_betweenness.head(10).to_string(index=False))



--- 가장 많이 인용되는 법령 TOP 10 (In-Degree) ---
            법령명  In-Degree
             민법       1327
공공기관의 운영에 관한 법률       1171
          전자정부법       1086
       개인정보 보호법       1050
          고등교육법        998
             형법        917
   개인정보 보호법 시행령        626
        초ㆍ중등교육법        525
             상법        507
            건축법        498


--- 영향력이 가장 높은 법령 TOP 10 (PageRank) ---
            법령명  PageRank
             형법  0.026341
             민법  0.026082
공공기관의 운영에 관한 법률  0.015025
          민사소송법  0.014460
          고등교육법  0.012180
          민사집행법  0.011820
             상법  0.011486
         국가공무원법  0.010215
       개인정보 보호법  0.008357
          전자정부법  0.007259


--- '허브' 역할을 하는 법령 TOP 10 (Betweenness) ---
                           법령명  Betweenness
                      지방세특례제한법     0.054351
                  대기환경보전법 시행규칙     0.046298
           기업활동 규제완화에 관한 특별조치법     0.046205
              산업입지 및 개발에 관한 법률     0.044713
                   조세특례제한법 시행령     0.035525
                      

### 커뮤니티 탐지

In [7]:
import networkx as nx
from networkx.algorithms import community
import pandas as pd

# 이전에 생성한 네트워크 그래프 G가 있다고 가정합니다.
# 만약 없다면, 이전 단계의 코드를 실행하여 G를 먼저 생성해야 합니다.
# G = nx.from_pandas_edgelist(df_edges, source='source', target='target', edge_attr='weight', create_using=nx.DiGraph())

print("네트워크 커뮤니티 탐지를 시작합니다 (루뱅 알고리즘)...")

# 1. 커뮤니티 탐지 실행
# 방향성이 없는 그래프로 변환하여 커뮤니티 탐지 (더 일반적)
G_undirected = G.to_undirected()
communities = community.louvain_communities(G_undirected, weight='weight')
print(f"총 {len(communities)}개의 커뮤니티를 발견했습니다.")

# 2. 각 노드가 어떤 커뮤니티에 속하는지 매핑
community_map = {}
for i, comm in enumerate(communities):
    for law_name in comm:
        community_map[law_name] = i

# 3. 결과를 데이터프레임으로 변환
df_community = pd.DataFrame(community_map.items(), columns=['법령명', '커뮤니티ID'])

# 4. 변호사 시험 과목 정보와 병합 (비교 분석을 위해)
try:
    public_laws = pd.read_csv('변호사시험_출제대상_부속법령(공법).csv', sep='\t')['법령명'].tolist()
    civil_laws = pd.read_csv('변호사시험_출제대상_부속법령(민사법).csv', sep='\t')['법령명'].tolist()
    criminal_laws = pd.read_csv('변호사시험_출제대상_부속법령(형사법).csv', sep='\t')['법령명'].tolist()

    def get_subject(law):
        if law in public_laws: return '공법'
        if law in civil_laws: return '민사법'
        if law in criminal_laws: return '형사법'
        return '기타'

    df_community['과목'] = df_community['법령명'].apply(get_subject)
    print("\n커뮤니티 분석 결과와 변호사 시험 과목 정보를 병합했습니다.")

except FileNotFoundError:
    print("\n(참고) 변호사 시험 과목 파일을 찾을 수 없어 과목 정보는 추가되지 않았습니다.")


# 5. 결과 확인
print("\n--- 커뮤니티 분석 결과 (상위 10개) ---")
print(df_community.head(10))

# 각 커뮤니티의 크기와 주요 과목 분포 확인
print("\n--- 각 커뮤니티별 크기 및 과목 분포 ---")
print(df_community.groupby('커뮤니티ID')['과목'].value_counts().unstack(fill_value=0))

# 특정 커뮤니티에 속한 법령 목록 확인 (예: 0번 커뮤니티)
print("\n--- 0번 커뮤니티에 속한 법령 목록 (일부) ---")
print(df_community[df_community['커뮤니티ID'] == 0]['법령명'].tolist()[:15])

네트워크 커뮤니티 탐지를 시작합니다 (루뱅 알고리즘)...
총 193개의 커뮤니티를 발견했습니다.

커뮤니티 분석 결과와 변호사 시험 과목 정보를 병합했습니다.

--- 커뮤니티 분석 결과 (상위 10개) ---
                                      법령명  커뮤니티ID  과목
0  4ㆍ16세월호참사 진상규명 및 안전사회 건설 등을 위한 특별법 시행령       0  기타
1               4차산업혁명위원회의 설치 및 운영에 관한 규정       1  기타
2                     각병과사병의군사경찰직무보조에관한규정       2  기타
3                        감사원규칙의 공포에 관한 규칙       3  기타
4                          감사원 정책자문위원회 규칙       4  기타
5               거창사건등 관련자의 명예회복에 관한 특별조치법       5  기타
6                            검사 선서에 관한 규정       6  기타
7                           검사의 법복에 관한 규칙       7  기타
8            경기도 수원시와 용인시의 관할구역 변경에 관한 규정       8  기타
9            경기도 수원시와 화성시의 관할구역 변경에 관한 규정       9  기타

--- 각 커뮤니티별 크기 및 과목 분포 ---
과목      공법  기타  민사법  형사법
커뮤니티ID                  
0        0   1    0    0
1        0   1    0    0
2        0   1    0    0
3        0   1    0    0
4        0   1    0    0
...     ..  ..  ...  ...
188      0   1    0    0
189      0   1    0    0
190      

In [8]:
# --- 6. 최종 결과 저장 (이 부분을 추가) ---
output_filename = '커뮤니티 탐지(현역).csv'
df_community.to_csv(output_filename, index=False, encoding='utf-8-sig')

In [ ]:
# from bertopic import BERTopic
# from sklearn.feature_extraction.text import CountVectorizer
# from konlpy.tag import Okt
# import os
# # --- 데이터 준비 ---
# # 1. 전처리된 텍스트 로드
# preprocessed_text_dir = '법령_전처리_텍스트'
# docs = []
# law_names_for_topic = []
# for filename in os.listdir(preprocessed_text_dir):
#     if filename.endswith('.txt'):
#         law_names_for_topic.append(filename[:-4])
#         with open(os.path.join(preprocessed_text_dir, filename), 'r', encoding='utf-8') as f:
#             docs.append(f.read())

# # 2. 분석 대상 그룹 정의 (변호사 시험 과목 기준)
# # (public_laws, civil_laws, criminal_laws 리스트가 메모리에 있다고 가정)
# law_to_subject = {law: '공법' for law in public_laws}
# law_to_subject.update({law: '민사법' for law in civil_laws})
# law_to_subject.update({law: '형사법' for law in criminal_laws})

# # --- 토픽 모델링 실행 ---
# # 1. 한국어 명사 추출을 위한 벡터라이저 설정
# okt = Okt()
# vectorizer = CountVectorizer(tokenizer=okt.nouns, max_features=3000)

# # 2. BERTopic 모델 초기화
# # min_topic_size: 토픽을 구성하기 위한 최소 문서(법령) 수
# topic_model = BERTopic(embedding_model="jhgan/ko-sbert-nli",
#                        vectorizer_model=vectorizer,
#                        min_topic_size=5,
#                        verbose=True)

# # 3. 모델 학습 및 토픽 추출
# # 시간이 매우 오래 걸릴 수 있습니다. ⏳
# topics, probs = topic_model.fit_transform(docs)

# # 4. 결과 확인
# print("\n--- BERTopic 분석 결과 ---")
# # 가장 자주 등장하는 토픽들
# print(topic_model.get_topic_info())

# # 특정 토픽의 키워드 확인 (예: 0번 토픽)
# print("\n--- 0번 토픽의 주요 키워드 ---")
# print(topic_model.get_topic(0))

# # --- 그룹별 토픽 분석 ---
# # 1. 각 법령에 토픽 ID와 과목 정보 추가
# df_topics = topic_model.get_document_info(docs, law_names_for_topic)
# df_topics['과목'] = df_topics['Name'].map(law_to_subject).fillna('기타')

# # 2. 각 과목 그룹별로 어떤 토픽이 주로 나타나는지 시각화
# # 이 시각화는 Jupyter Notebook 환경에서 가장 잘 보입니다.
# fig = topic_model.visualize_topics_by_class(
#     df_topics,
#     top_n_topics=10, # 상위 10개 토픽만 시각화
#     custom_labels=True
# )
# fig.show() # 대화형 차트 출력

c:\Users\silve\miniconda3\envs\bylaw\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-01 20:26:36,201 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 47/47 [00:02<00:00, 18.02it/s]
2025-08-01 20:26:59,666 - BERTopic - Embedding - Completed ✓
2025-08-01 20:26:59,666 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-08-01 20:27:06,259 - BERTopic - Dimensionality - Completed ✓
2025-08-01 20:27:06,262 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-08-01 20:27:06,280 - BERTopic - Cluster - Completed ✓
2025-08-01 20:27:06,280 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-08-01 20:29:05,455 - BERTopic - Representation - Completed ✓



--- BERTopic 분석 결과 ---
    Topic  Count               Name  \
0      -1    298     -1_경우_사항_해당_청구   
1       0     47      0_금융_투자_회사_집합   
2       1     45  1_해양수산부_어업_수산물_수산   
3       2     39    2_학교_교육_교육감_교육부   
4       3     35     3_기금_보조금_재정_회계   
..    ...    ...                ...   
80     79      6    79_건설_기계_승강기_공사   
81     80      6   80_조합_중앙회_부실_조합원   
82     81      6    81_도서관_대학_자료_학교   
83     82      6     82_보험_회사_계약_손해   
84     83      5   83_건축_건축물_건축사_해체   

                                       Representation  \
0            [경우, 사항, 해당, 청구, 개정, 장관, 기관, 호의, 다음, 산업]   
1            [금융, 투자, 회사, 집합, 증권, 은행, 신탁, 발행, 주식, 거래]   
2   [해양수산부, 어업, 수산물, 수산, 식품, 어촌, 농림축산식품부, 인증, 농산물,...   
3     [학교, 교육, 교육감, 교육부, 교원, 학생, 평생교육, 대학, 학교법인, 유치원]   
4           [기금, 보조금, 재정, 회계, 지출, 운용, 지방, 후원, 교부, 예산]   
..                                                ...   
80    [건설, 기계, 승강기, 공사, 발주, 수급인, 국토교통부, 하도급, 부품, 사업자]   
81         [조합, 중앙회, 부실, 조합원, 예금, 기금, 총회, 임원, 정관, 관

TypeError: list indices must be integers or slices, not str